In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the model
model = load_model(r"D:\modelv1.h5")

# Print the model summary
print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 512)         │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 6, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 3, 3, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 4,348,681 (16.59 MB)

 Trainable params: 4,344,711 (16.57 MB)

 Non-trainable params: 3,968 (15.50 KB)

 Optimizer params: 2 (12.00 B)

None


In [3]:
import requests

# URL of the haarcascade xml file
url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

# Download the file
response = requests.get(url)
with open("haarcascade_frontalface_default.xml", "wb") as file:
    file.write(response.content)


In [16]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the face cascade
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Load the model
model = tf.keras.models.load_model(r"D:\modelv1.h5")

# Function to preprocess images
def preprocess_image(img_path):
    # Load the image
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    if len(faces) == 0:
        raise ValueError("No faces detected in the image")

    # Crop the first detected face
    (x, y, w, h) = faces[0]
    face = gray[y:y+h, x:x+w]

    # Resize the face to 48x48
    face_resized = cv2.resize(face, (48, 48))
    
    # Expand dimensions to match the expected input shape
    face_resized = np.expand_dims(face_resized, axis=-1)  # Add channel dimension
    face_resized = np.expand_dims(face_resized, axis=0)   # Add batch dimension
    
    # Normalize the image
    face_resized = face_resized / 255.0
    
    return face_resized

# Function to predict the class of an image
def predict_image(model, img_path):
    img = preprocess_image(img_path)
    predictions = model.predict(img)
    return predictions

# Example usage
if __name__ == "__main__":
    # Path to the image
    img_path = r"D:\brave downloads\VIT Downloads\Other Downloads\archive (4)\test\disgust\PrivateTest_53395751.jpg"
    # Get predictions
    try:
        predictions = predict_image(model, img_path)
        print('Predictions:', predictions)
        # Optionally, get the class with the highest probability
        predicted_class = np.argmax(predictions, axis=1)
        print('Predicted class:', predicted_class)
        label=['angry','disgust','fear','happy','neutral','sad','surprise']
        print(label[predicted_class[0]])
    except ValueError as e:
        print(e)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
Predictions: [[0.29036915 0.22453423 0.1322419  0.07317881 0.02498978 0.18025884
  0.07442726]]
Predicted class: [0]
angry
